# TP1 - Branch & Bound, Cut-Generation for the TSP

## 1. The `minilp` python package

The `minilp` python package is a small python package that allows us to easily model (integer) linear program. The package comes with an interface to common linear programming solvers (`cplex`, `scipy`) but no integer linear programming solver.

<div class="alert alert-info">

The `minilp` module has been implemented for these practical sessions so you will not find any relevant documentation on Google. 
To get help on the module and its components, you can use the built-in `help` function:
        
    help(minilp)
    
</div>

In [1]:
import minilp
help(minilp)

Help on package minilp:

NAME
    minilp

PACKAGE CONTENTS
    exprs
    modeler
    problems
    results
    solvers

CLASSES
    abc.ABC(builtins.object)
        minilp.solvers.solver
    builtins.object
        minilp.exprs.cons
        minilp.exprs.expr
            minilp.exprs.var
        minilp.results.result
    enum.Enum(builtins.object)
        minilp.results.status
    minilp.modeler.modeler(builtins.object)
        minilp.problems.problem
    
    class cons(builtins.object)
     |  cons(lhs: 'expr', cmp: 'comparison_operator', rhs: 'expr')
     |  
     |  Class representing a linear constraint.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, lhs: 'expr', cmp: 'comparison_operator', rhs: 'expr')
     |      Args:
     |        - lhs: Left-hand side expression of the constraint.
     |        - cmp: Comparison operator of the constraint.
     |        - rhs: Right-hand side expression of the constraint.
     |  
     |  __repr__(self)
     |      Retur

In [2]:
import minilp

lp = minilp.problem("My first LP problem")

# Create two continuous variables within [0, 4]:
x1, x2 = lp.continuous_var_list(2, 0, 4)

# Add constraints:
lp.add_constraint(-3 * x1 + 4 * x2 <= 7)
lp.add_constraint(2 * x2 <= 5)
lp.add_constraint(6 * x1 + 4 * x2 <= 25)
lp.add_constraint(2 * x1 - x2 <= 6)

# Set the objective function:
lp.set_objective("max", x1 + 2 * x2)

# Solve the problem:
res = lp.lp_solve()
print(res)
print("x1 = {:.4f}, x2 = {:.4f}".format(res.get_value(x1), res.get_value(x2)))

status = OPTIMAL, obj. = 7.5
x1 = 2.5000, x2 = 2.5000


The `minilp` package also allows you to modelise simple integer linear programs.

<div class="alert alert-info alert-block">
    
There are also `binary_var` and `binary_var_list` method to create binary variable (integer variable constrained to 0 and 1).
The `_list` methods returns standard python `list` object, so you can combine them to create multi-dimensional lists of
minilp variables.
    
</div>

In [3]:
import minilp

lp = minilp.problem("My first ILP problem")

# Create two integer variables within [0, 4]:
x1, x2 = lp.integer_var_list(2, 0, 4)

# Add constraints:
lp.add_constraint(-3 * x1 + 4 * x2 <= 7)
lp.add_constraint(2 * x2 <= 5)
lp.add_constraint(6 * x1 + 4 * x2 <= 25)
lp.add_constraint(2 * x1 - x2 <= 6)

# Set the objective function:
lp.maximize(x1 + 2 * x2)

While `minilp` allows you to model (mixed-)integer linear program, it does not provide a integer linear program solver &mdash; the `lp_solve` method will always solve the linear relaxation of the problem.

<div class="alert alert-info alert-block">
    
You can use a different solver by passing a different object to the `lp_solve` method. Available solvers are:
    
- `minilp.solvers.scipy` &mdash; A solver based on the `scipy` module, wildly available.
- `minilp.solvers.docplex` &mdash; A solver based on the `docplex` module, which requires a valid CPLEX installation.
    
The default solver used is `docplex` if available, otherwize `scipy`.
    
</div>

In [4]:
res = lp.lp_solve()
print(res)
print("x1 = {:.4f}, x2 = {:.4f}".format(res.get_value(x1), res.get_value(x2)))

status = OPTIMAL, obj. = 7.5
x1 = 2.5000, x2 = 2.5000


The `minilp` package allows you to modelise `<=`, `>=` or `==` (in)equalities. You can create linear expression by simply adding, substracting or multiplying values (`int` or `float`) and variables or existing expressions. You can use the standard python `sum` to sum a bunch of expressions or variables, and the `minilp.dot` function to compute the [dot product](https://en.wikipedia.org/wiki/Dot_product) of two vectors.

**Exercice:** Complete the following code to create a simple model for the knapsack problem.

<div class="alert alert-block alert-info">
    
Make your program as generic as possible, i.e., if `N` or `K` changes, you should not have to
modify the code of your problem.
    
</div>

In [5]:
import minilp

N = 5
p = [1, 4, 5, 3, 5]  # profits
w = [3, 4, 3, 5, 9]  # weights
K = 10  # capacity
assert N == len(w) and N == len(p)

# A simple knapsack
kp = minilp.problem("Simple knapsack")

#Create variables, add constraints and set the objective.
x = kp.binary_var_list(N, 'x')

# Add constraints:
kp.add_constraint(minilp.modeler.modeler.dot(x,w) <= K)

# Set the objective function:
kp.maximize(minilp.modeler.modeler.dot(x,p))

# We can solve the linear relaxation:
res = kp.lp_solve()
print(res)
print(res.get_values(x))


status = OPTIMAL, obj. = 10.8
[0.0, 1.0, 1.0, 0.6000000000000001, 0.0]


## 2. Generic Branch & Bound

The purpose of the following section is to implement a generic branch-and-bound procedure based on a `minilp.problem` instance.

### 2.1. Node structure

<div class="alert alert-block alert-danger">
    
Do not overthink this section!  
The three methods you have to implement in this section (`create_root_node`, `create_root_node` and `is_empty`) are
all **one-liner** and can be implemented with **a single `return` statement**.
    
</div>

We are going to use a simple list to represent the current set of leaf nodes in the branch-and-bound tree.
For each node, we are only going to store the list of separation constraints of the node (and not an actual 
problem).

For instance, assume a problem with variables $x_1$ and $x_2$, we two separation constraints, our node
could be created with:

```python
node = [x1 <= 2, x2 >= 4]
```

In [6]:
import minilp
from typing import List, Optional

# minilp.cons is the Python type representing minilp constraints:
Node = List[minilp.cons]

**Question:** What does the root node of the branching tree contain?  
=> THE OPTIMUM OF THE RELAXED PROBLEM

**Exercice:** Implement the function `create_root_node()` that creates the root node of the branching tree.

In [7]:
def create_root_node() -> Node:
    """
    Creates the root node for a branch-and-bound procedure.

    Returns:
        The root node for a branch-and-bound procedure.
    """
    return []

For the sake of simplicity, we are going to process node in a first-in/first-out order.

**Exercice:** Implement the function `get_next_node(nodes)` that extract the next node to process from the tree.

<div class="alert alert-info alert-block">
    
You can check the `list.pop` method from python.
    
</div>

In [8]:
def extract_next_node(nodes: List[Node]) -> Node:
    """
    Extracts the next node to process from the given list of nodes. The node
    is removed from the given list before being returned.

    Args:
        nodes: Current list of nodes to extract the next node from.

    Return:
        The next node process.
    """
    return nodes.pop() 

**Exercice:** Implement the function `is_empty(nodes)` that returns `True` if there are no more nodes to process in the list.

In [9]:
def is_empty(nodes: List[Node]):
    """
    Checks if there are any nodes remaining to process in the given list.

    Args:
      - nodes: The list of nodes to check.

    Returns:
        True if there are no more nodes, False otherwise.
    """
    return not nodes 

### 2.2. Comparing `minilp.result`

The `minilp.problem.lp_solve` method returns an instance of `minilp.result`. The following method compare two `minilp.result` objects for a given 
problem and returns `True` if the left one is better.

<div class="alert alert-block alert-warning">
    
Don't forget to execute the cell even if you do not have to modify it in order to have access to the `compare_solution` function.
    
</div>

In [10]:
def compare_solutions(
    problem: minilp.problem, l: minilp.result, r: minilp.result
) -> bool:
    """
    Compares the two given solutions, returning True if the left
    one is better than the right one for the current problem.

    Args:
        problem: The problem for which the solution are compared.
        l, r: The two solutions (minilp.result) to compare.

    Returns:
        True if the left solution is better than the right one, or
        if the right solution has no solution.
    """
    if problem.isnan(r.objective):
        return True
    if problem.sense == "min":
        return l.objective < r.objective
    return l.objective > r.objective

### 2.3. Finding bound variable

In order to execute the branch-and-bound procedure, we must be able to find a variable to separate the problem on.
For this notebook, we are always going to separate on the first non-integer variable.

<div class="alert alert-block alert-info">
    
You can access the list of variables in the problem with `minilp.problem.variables` and their values in the
solution with `minilp.result.get_value` or `minilp.result.get_values`.
    
</div>

<div class="alert alert-block alert-warning">
    
There are variables in the problem that **do not have** to be integers in the solution (e.g., transport
variables in the warehouse allocation problem).
These variables should not be checked against when looking for a non-integer value.

You can retrieve the type of a variable (`int` or `float`) using `minilp.expr.var.category`.
    
</div>

**Exercire:** Implement the `get_first_non_integral(problem, result)` method that, given a problem and solution, returns
the first variable of the problem that is not integral in the result (and should be), or `None` if there is no such variable

<div class="alert alert-block alert-info">
    
In order to solve the linear relaxation of the problem, multiple matrix operations have to be performed. These 
operations are numerically imprecise, thus it is common to find near-integral values (`1.0000001` or `2.9999999`).
Such values should be considered integral for the purpose of our algorithms, as long as their distance to the
nearest integer is less than `eps` (default to `1e-6`).
    
</div>

In [11]:
def find_first_non_integral(
    problem: minilp.problem, result: minilp.result, eps: float = 1e-6
) -> Optional[minilp.var]:
    """
    Retrieves the first integer variable in the given problem
    whose value is not integral in the given solution.

    Args:
        problem: The problem to find a variable from.
        result: A solution of the problem.
        eps: The maximum allowed distance to consider a variable integral.

    Returns:
        The first variable (minilp.expr.var) whose value is not integral,
        or None if no such variable exists.
    """
    for i in range(len(problem.variables)):
        var = problem.variables[i]
        res = result.get_value(var)
        if (var.category is int and abs(res-round(res))>eps ):
            return var
    
    return None
            
#TEST
#print(find_first_non_integral(kp, kp.lp_solve()))

### 2.4. Relaxation, separation, iteration

**Exercice:** Implement the `iterate` method below that performs a single iteration of the branch-and-bound algorithm, i.e., extract a node, 
solve its relaxation, and then update the current best solution (return) or separate the problem. 

<div  class="alert alert-block alert-info">
    
You can use the `minilp.problem.add_constraints` and `minilp.problem.del_constraints` methods to add or remove constraints from
a `minilp.problem`.
    
Do not forget to return the `minilp.result` you found if it is your new best integer solution!
    
</div>

<div class="alert alert-block alert-danger">
    
Again, **do not overthink** this section! While `iterate` is the core function of the branch-and-bound algorith, it is a 
pretty simply and short function that can be implemented in about 10 lines.
    
</div>

In [12]:
import math


def iterate(
    problem: minilp.problem,
    current_best: minilp.result,
    nodes: List[Node],
    solver: minilp.solver = minilp.solvers.get_default_solver(),
    eps: float = 1e-6,
) -> Optional[minilp.result]:
    """
    Performs an iteration of the branch-and-bound algorithm.

    Args:
        problem: Problem to perform an iteration for.
        current_best: Current best known feasible (integral) solution. Instance of minilp.result.
        nodes: Current list of nodes to update.
        solver: Solver to use to solve the linear relaxation.
        eps: The maximum allowed distance to consider a variable integral.

    Returns:
        The new best solution if the current one was improved, or None if no new solution
        was found.
    """
    
    #extract a node    
    if is_empty(nodes):
        return None

    node = extract_next_node(nodes)
    
    problem.add_constraints(node)
    #solve its relaxation
    res = problem.lp_solve()
    problem.del_constraints(node)    
    
    isBetter = compare_solutions(problem, res, current_best)
    
    if isBetter: 
        var = find_first_non_integral(problem, res)
        if(var is None):
        #update the current best solution
            return res
        else:
        #or separate the problem
            node_left = node.copy()
            node_left.append(var <= math.floor(res.get_value(var)))
            node_right = node.copy()
            node_right.append(var >= math.ceil(res.get_value(var)))
            nodes.append(node_left)
            nodes.append(node_right)
            return None
    else:
        return None
    

### 2.5. The branch-and-bound algorithm

The cell below defines the global `branch_and_bound` procedure.

In [13]:
import datetime as dt


def log_solution(
    niterations: int, nodes: List[Node], res: minilp.result, new_best: bool = False
):
    """
    Logs the given solution with time information.

    Args:
        niterations: Number of iterations.
        nodes: List of nodes.
        res: Solution (minilp.result) to log.
        new_best: Indicates if this solution is new best solution (integer).
    """
    print(
        "{} {:5d} {:5d} {:9g}{}".format(
            dt.datetime.now().strftime("%T"),
            niterations,
            len(nodes),
            res.objective,
            "*" if new_best else "",
        )
    )


def branch_and_bound(
    problem: minilp.problem,
    lp_solver: minilp.solver = minilp.solvers.get_default_solver(),
    eps: float = 1e-6,
    log_frequency: int = 10,
) -> minilp.result:
    """
    Applies a branch-and-bound algorithm to solve the given problem.

    Args:
        problem: A minilp.problem instance corresponding to the problem to solve.
        lp_solver: Solver to use to solve the linear relaxation.
        eps: The maximum allowed distance to consider a variable integral.
        log_frequency: Number of iterations between two log (not including exceptional log).

    Returns:
        A minilp.result containing the result of running the branch-and-bound
        algorithm.
    """

    print(
        "B&B using {} to solve linear relaxation".format(lp_solver.__class__.__name__)
    )

    # Insert the first node in the list:
    nodes = []
    nodes.append(create_root_node())

    # Current best result (unknown):
    current_best = minilp.result()

    # Counter for iterations:
    nb_iterations = 0

    while not is_empty(nodes):

        # Increment counter and log.
        if nb_iterations % log_frequency == 0:
            log_solution(nb_iterations, nodes, current_best)

        nb_iterations += 1

        # Iterate:
        new_best = iterate(problem, current_best, nodes, lp_solver, eps)

        # Check if we have a new current best:
        if new_best is not None:
            current_best = new_best
            log_solution(nb_iterations, nodes, current_best, True)

    # Return the best solution found (if any).
    return current_best

**Exercice:** Use the `branch_and_bound` method to solve the knapsack instance defined at the beginning of the notebook.

In [14]:
import minilp

N = 5
p = [1, 4, 5, 3, 5]  # profits
w = [3, 4, 3, 5, 9]  # weights
z = [0, 0, 0, 0, 0]
K = 10  # capacity

assert N == len(w) and N == len(p)

# A simple knapsack
kp = minilp.problem("Simple knapsack")

# TODO: Create variables, add constraints and set the objective.
x = kp.binary_var_list(N)
kp.add_constraint(kp.dot(x, w) <= K)

kp.maximize(kp.dot(x, p))

# We can solve the linear relaxation:
res = branch_and_bound(kp)
print(res)
print(res.get_values(x))

B&B using scipy to solve linear relaxation
15:35:23     0     1       nan
15:35:23     4     3       -12*
15:35:23     8     3         0*
15:35:23     9     2         7*
15:35:23    10     3         7
15:35:23    16     1         8*
15:35:23    19     0        10*
status = OPTIMAL, obj. = 10.0
[1.0, 1.0, 1.0, 0.0, 0.0]


**Exercice:** Create other instances of the knapsack problem to reach the "limits" of your implementation &mdash; What is the largest instance you can solve in e.g. less than 5 seconds?

<div class="alert alert-block alert-info">
    
You can use the `numpy.random` module to generate arrays of random integer or floating point values. 
    
</div>

In [15]:
# The largest we can find in less than 5 seconds is N = 20
# It also depends on the seed, but a number between 18 and 20 seems like a good value for the instance size.

In [16]:
import minilp
import numpy as np

N = 18
p = np.random.randint(50, 100, size=N)
w = np.random.randint(20, 40, size=N)
K = np.sum(w) // (1.5 + np.random.rand() * 1)

assert N == len(w) and N == len(p)

print("Knapsack problem with {} items and a capacity of {}.".format(N, K))
print("  Profits: {}".format(p))
print("  Weights: {}".format(w))

# A simple knapsack
kp = minilp.problem("Simple knapsack")

# TODO: Create variables, add constraints and set the objective.
x = kp.binary_var_list(N)
kp.add_constraint(kp.dot(x, w) <= K)

kp.maximize(kp.dot(x, p))

# We can solve the linear relaxation:
res = branch_and_bound(kp, log_frequency=5)
print(res)
print(res.get_values(x))

Knapsack problem with 18 items and a capacity of 208.0.
  Profits: [52 81 99 99 64 94 93 95 81 73 61 95 81 80 68 61 84 97]
  Weights: [22 26 27 37 25 28 39 23 21 38 21 30 30 24 26 30 30 28]
B&B using scipy to solve linear relaxation
15:35:23     0     1       nan
15:35:23     5     6       nan
15:35:23    10     9      -783*
15:35:23    10     9      -783
15:35:23    12     9      -772*
15:35:23    14     9      -769*
15:35:23    15    10      -769
15:35:23    18     9      -756*
15:35:23    20     9      -752*
15:35:23    20     9      -752
15:35:23    24     9      -740*
15:35:23    25    10      -740
15:35:23    28     9         0*
15:35:23    29     8       688*
15:35:23    30     9       688
15:35:24    35     8       688
15:35:24    40     9       688
15:35:24    45     8       688
15:35:24    48     7       702*
15:35:24    50     7       702
15:35:24    55     8       702
15:35:24    60     7       702
15:35:24    65     6       702
15:35:24    70     7       702
15:35:24    75

## 3. The Travelling Salesman Problemn (TSP)


Given a list of $n$ cities and the distances $c_{ij}$ between each pair of cities, you want to find the shortest circuit that visits each city **exactly once** and comes back to the first visited city.

<div class="alert alert-info alert-block">

The `tsp.data` packages contains grid of distances of various sizes (5, 6, 7, 8, 9, 10, 15, 17, 26, 42).

</div>

The goal of this section is to implement a cut-generation algorithm for the travelling salesman problem, using the `minilp` python package.

### 3.1. Creating a model for the TSP

### 3.1.1. Relaxation of the TSP

We call "TSP relax" the relaxation of the TSP problem that do not include constraints to eliminate subtours. 
The model is given below:

  - $x_{ij}\in\{0, 1\}$ &mdash; Binary variable indicating if we go directly from city $i$ to city $j$.

$
\begin{align}
  \text{min.} \quad & \sum_{i = 1}^{n} \sum_{j=1}^{n} c_{ij}x_{ij} & \\
  \text{s.t.} \quad & \sum_{\substack{j = 1\\ j \neq i}}^{n} x_{ij} = 1, & \forall i\in \left\{1,~\ldots,~n\right\}\\
                    & \sum_{\substack{i = 1\\i \neq j}}^{n} x_{ij} = 1, & \forall j\in \left\{1,~\ldots,~n\right\}\\
                    & x_{ij} \in\{0,1\}, & \forall i,j \in \{1,\ldots,n\}
\end{align}
$
    
**Exercice:** Create the `tsp_relax` function that creates a `minilp.problem` instance corresponding to
the relaxed TSP problem.

In [17]:
from typing import Sequence, Tuple
import minilp


def tsp_relax(
    distances: Sequence[Sequence[float]], name: str = "TSP relax"
) -> Tuple[minilp.problem, Sequence[Sequence[minilp.var]]]:
    """
    Create a 'relaxed' model for the TSP. A relaxed includes all the standard
    constraints of the TPS, but not the subtours constraints.

    Args:
        distances: The matrix of distances (diagonal is zeros).
        name: The name of the model.

    Returns:
        A tuple (model, vars) where model is the TSP model and vars is the matrix
        of x variables.
    """

    N = len(distances)
    c = distances

    tsp = minilp.problem(name)
    
    #Create variables, add constraints and set the objective.
    x = [tsp.binary_var_list(N, f"x{i}_") for i in range(N)]
    #for i in range(len(x)):
    #    del (x[i][i])

    #tc  = []
    #for d in c:
    #    for e in d:
    #       tc.append(e) 
    
    #tx  = []
    #for d in x:
    #    for e in d:
    #        tx.append(e) 

    # Add constraints:
    #for d in x :
    #    tsp.add_constraint(tsp.sum(d) == 1)
    #for i in range(len(x[0])):
    #    tsp.add_constraint(tsp.sum([row[i] for row in x]) == 1)
    #tsp.add_constraint(tsp.sum(x[,:]) == 1)
    # Set the objective function:
    
    for i in range(N):
        tsp.add_constraint(sum(x[i][j] for j in range(N) if i != j) == 1)
        
    for i in range(N):
        tsp.add_constraint(sum(x[j][i] for j in range(N) if i != j) == 1)
    
    tsp.minimize(sum(x[i][j]*distances[i][j] for i in range(N) for j in range(N)))
    
    for i in range(N):
        tsp.add_constraint(x[i][i]==0)
    # Returns both the problem and the variables:
    return tsp, x

<div class="alert alert-info alert-block">
    
You can visualize a `minilp` model by using `repr(model)` or simply writting the variable name at the end of a cell.

</div>

In [18]:
import tsp.data as data

tsp, x = tsp_relax(data.grid5)
tsp


ILP --- TSP relax
-----------------
min.   3 * x0_1 + 4 * x0_2 + 2 * x0_3 + 7 * x0_4 + 3 * x1_0 + 4 * x1_2 + 6 * x1_3 + 3 * x1_4 + 4 * x2_0 + 4 * x2_1 + 5 * x2_3 + 8 * x2_4 + 2 * x3_0 + 6 * x3_1 + 5 * x3_2 + 6 * x3_4 + 7 * x4_0 + 3 * x4_1 + 8 * x4_2 + 6 * x4_3
s.t.   x0_1 + x0_2 + x0_3 + x0_4 == 1
       x1_0 + x1_2 + x1_3 + x1_4 == 1
       x2_0 + x2_1 + x2_3 + x2_4 == 1
       x3_0 + x3_1 + x3_2 + x3_4 == 1
       x4_0 + x4_1 + x4_2 + x4_3 == 1
       x1_0 + x2_0 + x3_0 + x4_0 == 1
       x0_1 + x2_1 + x3_1 + x4_1 == 1
       x0_2 + x1_2 + x3_2 + x4_2 == 1
       x0_3 + x1_3 + x2_3 + x4_3 == 1
       x0_4 + x1_4 + x2_4 + x3_4 == 1
       x0_0 == 0
       x1_1 == 0
       x2_2 == 0
       x3_3 == 0
       x4_4 == 0

### 3.1.2. MTZ formulation for the TSP

We call "TSP relax" the relaxation of the TSP problem that do not include constraints to eliminate subtours. 
The model is given below:

  - $x_{ij}\in\{0, 1\}$ &mdash; Binary variable indicating if we go directly from city $i$ to city $j$.
  - $u_{i} \in\{1,\ldots,n\}$ &mdash; Subtour elimination variables: $u_i$ is the position of
    city $i$ in the tour.

$
\begin{align}
  \text{min.} \quad & \sum_{i = 1}^{n} \sum_{j=1}^{n} c_{ij}x_{ij} & \\
  \text{s.t.} \quad & \sum_{\substack{j = 1\\ j \neq i}}^{n} x_{ij} = 1, & \forall i\in \left\{1,~\ldots,~n\right\}\\
                    & \sum_{\substack{i = 1\\i \neq j}}^{n} x_{ij} = 1, & \forall j\in \left\{1,~\ldots,~n\right\}\\
                    & u_1 = 1 & \\
                    & 2 \leq u_i \leq n, & \forall i\in\{2,\ldots,n\} \\
                    & u_i - u_j +1 \leq (n-1)(1 - x_{ij}), &\forall i, j \in \{2,\ldots,n\}\\
                    & x_{ij} \in\{0,1\},\ u_{i}\in\mathbb{N} & \forall i,j \in \{1,\ldots,n\}
\end{align}
$
    
**Exercice:** Create the `tsp_mtz` function that creates a `minilp.problem` instance corresponding to
the MTZ formulation of the TSP.

<div class="alert alert-info alert-block">
    
Use the `tsp_relax` method to initialize the problem with the basic TSP constraints and retrieve the $x$ variables.
    
</div>

In [19]:
import minilp


def tsp_mtz(
    distances: Sequence[Sequence[float]], name: str = "TSP MTZ"
) -> Tuple[minilp.problem, Tuple[Sequence[Sequence[minilp.var]], Sequence[minilp.var]]]:
    """
    Create a MTZ model for the TSP.

    Args:
        distances: The matrix of distances (diagonal is zeros).
        name: The name of the model.

    Returns:
        A tuple (model, (x, u)) where model is the TSP model and (x, u) are the
        variables of the model (x is a 2D array, u is a 1D array).
    """

    N = len(distances)
    

    # Use tsp_relax to create the basic model:
    tsp, x = tsp_relax(distances, name)

    u = tsp.integer_var_list(N, prefix="u")
    print(u)
    tsp.add_constraint(u[0]== 1)
    for i in range(1, len(u)):
        tsp.add_constraint(u[i]>= 2)
        tsp.add_constraint(u[i]<= N)
    

    for i in range(1,N):
        for j in range(1,N):
            tsp.add_constraint(u[i] - u[j] + 1 <= (N-1) * (1-x[i][j]))

    # Returns both the problem and the variables:
    return tsp, (x, u)

In [ ]:
import tsp.data as data

tsp, x = tsp_mtz(data.grid5)
tsp



### 3.1.3. Flow formulation for the TSP

We call "TSP relax" the relaxation of the TSP problem that do not include constraints to eliminate subtours. 
The model is given below:

  - $x_{ij}\in\{0, 1\}$ &mdash; Binary variable indicating if we go directly from city $i$ to city $j$.
  - $y_{ij} \in\mathbb{R}_*^+$ &mdash; Subtour elimination variables: $y_{ij}$ is the *flow* on arc $(i,j)$, each city
    produces one unit of flow.

$
\begin{align}
  \text{min.} \quad & \sum_{i = 1}^{n} \sum_{j=1}^{n} c_{ij}x_{ij} & \\
  \text{s.t.} \quad & \sum_{\substack{j = 1\\ j \neq i}}^{n} x_{ij} = 1, & \forall i\in \left\{1,~\ldots,~n\right\}\\
                    & \sum_{\substack{i = 1\\i \neq j}}^{n} x_{ij} = 1, & \forall j\in \left\{1,~\ldots,~n\right\}\\
                    & \sum_{j=2}^{n} y_{1j} = 1 & \\
                    & \sum_{j=1}^{n} y_{ij} = \sum_{j=1}^{n} y_{ji} + 1, & \forall i\in\{2,\ldots,n\} \\
                    & y_{ij}  \leq n x_{ij}, &\forall i, j \in \{1,\ldots,n\}\\
                    & x_{ij} \in\{0,1\}, y_{ij}\in\mathbb{R}_*^{+} & \forall i,j \in \{1,\ldots,n\}
\end{align}
$
    
**Exercice:** Create the `tsp_flow` function that creates a `minilp.problem` instance corresponding to
the flow formulation of the TSP.

<div class="alert alert-info alert-block">
    
Use the `tsp_relax` method to initialize the problem with the basic TSP constraints and retrieve the $x$ variables.
    
</div>

In [21]:
import minilp


def tsp_flow(
    distances: Sequence[Sequence[float]], name: str = "TSP Flow"
) -> Tuple[
    minilp.problem,
    Tuple[Sequence[Sequence[minilp.var]], Sequence[Sequence[minilp.var]]],
]:
    """
    Create a Flow model for the TSP.

    Args:
        distances: The matrix of distances (diagonal is zeros).
        name: The name of the model.

    Returns:
        A tuple (model, (x, y)) where model is the TSP model and (x, y) are the
        variables of the model (x and y are 2D arrays).
    """

    N = len(distances)

    # Use tsp_relax to create the basic model:
    tsp, x = tsp_relax(distances, name)
    
        
    #Create variables, add constraints and set the objective.
    y = [tsp.binary_var_list(N, f"y{i}_") for i in range(N)]
    
    for yi in y:
        tsp.add_constraint(sum(x[i][j] for j in range(N) if i != j) == 1)

    ...  # TODO

    # Returns both the problem and the variables:
    return tsp, (x, y)

### 3.2. Solving small instances of the TSP problem

**Exercice:** Using the `tsp_mtz`, `tsp_flow` and `branch_and_bound` functions you implemented, solve the **small** TSP instances found in `tsp.data`.

**Question:** How large are the instances you are able to solve in a reasonable amount of time?

In [ ]:
import tsp.data as data

distances = data.grid5

print("=== MTZ ===")
tsp, (x, u) = tsp_mtz(distances)
#print(repr(tsp))
r = branch_and_bound(tsp)
print(r)

print("=== Flow ===")
#tsp, (x, y) = tsp_flow(distances)
#r = branch_and_bound(tsp, eps=1e-8, log_freqeuncy=2)
print(r)

### 3.3. Cut generation algorithm for the TSP

We are going to implement a cut-generation procedure using the following TSP formulation:

  - $x_{ij}\in\{0, 1\}$ &mdash; Binary variable indicating if we go directly from city $i$ to city $j$.

$
\begin{align}
  \text{min.} \quad & \sum_{i = 1}^{n} \sum_{j=1}^{n} c_{ij}x_{ij} & \\
  \text{s.t.} \quad & \sum_{\substack{j = 1\\ j \neq i}}^{n} x_{ij} = 1, & \forall i\in \left\{1,~\ldots,~n\right\}\label{tsp2:leave}\\
                    & \sum_{\substack{i = 1\\i \neq j}}^{n} x_{ij} = 1, & \forall j\in \left\{1,~\ldots,~n\right\}\label{tsp2:enter}\\
                    & \sum_{i\in S} \sum_{j \notin S} x_{ij} \geq 1 & \mathcal{S} \subset \{1, \ldots, n\},\ S \ne \emptyset \label{tsp4:subtour:0} \\
                    & x_{ij} \in\{0,1\}, & \forall i,j \in \{1,\ldots,n\}
\end{align}
$

Where $\mathcal{S}$ is the set of subtours in the graph. 
The generation procedure is as follow:

1. We start with an empty set $\mathcal{S} = \emptyset$.
2. We solve the problem (completely) using our `branch_and_bound` procedure.
3. We find all the subtours in the solution:
 - If there is a single subtours, we have found the optimal solution.
 - Otherwize, we add the subtours to the set $\mathcal{S}$ and we got back to 2.

#### 3.3.1. Generating subtour constraints

**Exercice:** Implement the `add_subtour_constraints` method that, given a list of subtours (a list of list of integers, e.g., `[1, 2, 4]` is a subtour going through nodes 1, 2 and 4),
create subtour constraints and add them to the given problem.

In [23]:
def add_subtour_constraints(
    tsp: minilp.problem,
    x: Sequence[Sequence[minilp.var]],
    subtours: Sequence[Sequence[int]],
) -> Sequence[minilp.cons]:
    """
    Generates and adds subtours constraints for the given problem.

    Args:
        tsp: The current TSP problem.
        x: The x variables (2D-array) of the TSP formulation.
        subtours: The list of subtours. A subtour is a list of int containing the ID of the node.

    Returns:
        The generated constraints.
    """
    ...  # TODO


# We can test the method by generated a relaxed-TSP and asking constraints for random subtours:
tsp, x = tsp_relax(data.grid5)
add_subtour_constraints(tsp, x, [[1, 2, 3], [0, 4]])

#### 3.3.2. Finding subtours in a solution

**Exercice:** Implement the method `find_subtours` that given a 2D-array `x` of integer (0 or 1), returns a list of subtours in `x`.

In [24]:
def find_subtours(x: Sequence[Sequence[bool]]) -> Sequence[Sequence[int]]:
    """
    Extracts subtours from the given 2D-array.

    Args:
        x: A two-dimensional array corresponding to the x variable in the TSP formulation, where
            x[i][j] is 1 if arc (i, j) is used.

    Returns:
        A list of subtours, where each subtour is a list.
    """
    ...  # TODO


# We can check the method by using a custom x array corresponding
# to two subtours (0 -> 2 -> 0) and (1 -> 4 -> 3 -> 1):
find_subtours(
    [
        [0, 0, 1, 0, 0],  # 0 -> 2
        [0, 0, 0, 0, 1],  # 1 -> 4
        [1, 0, 0, 0, 0],  # 2 -> 0
        [0, 1, 0, 0, 0],  # 3 -> 1
        [0, 0, 0, 1, 0],  # 4 -> 3
    ]
)

#### 3.3.3. Branch-and-cut for the TSP

**Exercice:** Complete the `tsp_branch_and_cut` method below.

In [25]:
from typing import Callable


def tsp_branch_and_cut(
    distances: Sequence[Sequence[float]],
    ilp_solver: Callable[[minilp.problem], minilp.result] = branch_and_bound,
):
    """
    Solves the given TSP instance using a branch-and-cut with the given solver.

    Args:
        distances: Transport costs for the TSP.
        ilp_solver: Function that can be called as ilp_solver(ilp) to solve integer linear program.

    Returns:
        A minilp.result solution for the given TSP instance.
    """
    # Create the relaxation:
    N = len(distances)
    tsp, x = tsp_relax(distances, "TSP Branch & Cut")

    while True:

        # Solve the problem:
        res = ilp_solver(tsp)

        ...  # TODO

    return res

**Exercice:** Test your `tsp_branch_and_cut` implementation.

In [ ]:
import tsp.data as data

res = tsp_branch_and_cut(data.grid5)

B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:35:25     0     1       nan
15:35:25     1     0        17*
B&B using scipy to solve linear relaxation
15:

15:35:26     1     0        17*
B&B using scipy to solve linear relaxation
15:35:26     0     1       nan
15:35:26     1     0        17*
B&B using scipy to solve linear relaxation
15:35:26     0     1       nan
15:35:26     1     0        17*
B&B using scipy to solve linear relaxation
15:35:26     0     1       nan
15:35:26     1     0        17*
B&B using scipy to solve linear relaxation
15:35:26     0     1       nan
15:35:27     1     0        17*
B&B using scipy to solve linear relaxation
15:35:27     0     1       nan
15:35:27     1     0        17*
B&B using scipy to solve linear relaxation
15:35:27     0     1       nan
15:35:27     1     0        17*
B&B using scipy to solve linear relaxation
15:35:27     0     1       nan
15:35:27     1     0        17*
B&B using scipy to solve linear relaxation
15:35:27     0     1       nan
15:35:27     1     0        17*
B&B using scipy to solve linear relaxation
15:35:27     0     1       nan
15:35:27     1     0        17*
B&B using scip

15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scipy to solve linear relaxation
15:35:28     0     1       nan
15:35:28     1     0        17*
B&B using scip

15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:29     1     0        17*
B&B using scipy to solve linear relaxation
15:35:29     0     1       nan
15:35:30     1     0        17*
B&B using scipy to solve linear relaxation
15:35:30     0     1       nan
15:35:30     1     0        17*
B&B using scipy to solve linear relaxation
15:35:30     0     1       nan
15:35:30     1     0        17*
B&B using scipy to solve linear relaxation
15:35:30     0     1       nan
15:35:30     1     0        17*
B&B using scip

15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scipy to solve linear relaxation
15:35:31     0     1       nan
15:35:31     1     0        17*
B&B using scip

...

15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scipy to solve linear relaxation
15:42:25     0     1       nan
15:42:25     1     0        17*
B&B using scip

15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scipy to solve linear relaxation
15:42:26     0     1       nan
15:42:26     1     0        17*
B&B using scip

15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scipy to solve linear relaxation
15:42:28     0     1       nan
15:42:28     1     0        17*
B&B using scip

15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scipy to solve linear relaxation
15:42:29     0     1       nan
15:42:29     1     0        17*
B&B using scip

15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scipy to solve linear relaxation
15:42:31     0     1       nan
15:42:31     1     0        17*
B&B using scip

<div class="alert alert-block alert-danger"></div>